This code aims to quantify the nuclei and the size and number of synuclein inclusions for single transfections with timecourse, live timecourse, fixed, and plurisin experiments.

Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import remove_small_objects, binary_dilation, disk
from skimage.measure import label, regionprops
from skimage import exposure, morphology

Define Sub Functions

In [2]:
def load_images_from_folder(folder_path):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

def preprocess_dapi_channel(image):
    """Extract and preprocess the DAPI channel from the image."""
    dapi_channel = image[0]
    blurred_dapi = gaussian(dapi_channel, sigma=2)  # Smooth the image with Gaussian blur
    return blurred_dapi

def segment_nuclei(blurred_dapi):
    """Segment the nuclei from the preprocessed DAPI channel."""
    threshold_value = threshold_otsu(blurred_dapi)  # Apply Otsu thresholding
    binary_image = blurred_dapi > threshold_value  # Convert to binary image
    cleaned_image = remove_small_objects(binary_image, min_size=400)  # Remove small objects
    return binary_dilation(cleaned_image, footprint=disk(5))  # Dilate the image to merge adjacent objects

def count_nuclei(labeled_image):
    """Count the number of nuclei in the labeled image."""
    return len(np.unique(labeled_image)) - 1  # Subtract 1 to exclude the background label

def preprocess_green_channel(image):
    """Extract and preprocess the green channel from the image."""
    green_channel = image[1]
    confocal_img = exposure.adjust_sigmoid(green_channel, cutoff=0.4)  # Adjust brightness/contrast
    confocal_img = (confocal_img - confocal_img.min()) / (confocal_img.max() - confocal_img.min())  # Normalize
    return confocal_img

def segment_inclusions(confocal_img, threshold_value=0.19):
    """Segment inclusions in the green channel using a fixed threshold."""
    return confocal_img > threshold_value  # Convert to binary image

def measure_inclusion_sizes(labeled_image, confocal_img):
    """Measure the size of each inclusion in the labeled image."""
    props = regionprops(labeled_image, confocal_img)
    return [prop.area for prop in props]  # Extract and return the area of each region

def append_to_results(image_sizes, sizes_df, path, iteration):
    """Append the inclusion sizes to the DataFrame for all images."""
    sizes_df_add = pd.DataFrame(image_sizes, columns=[f'image {path}'])
    if iteration == 1:
        return sizes_df_add
    return pd.concat([sizes_df, sizes_df_add], axis=1)

def clean_up_dataframe(sizes_df):
    """Clean up the DataFrame by replacing 1s with NaN and shifting values."""
    sizes_df = sizes_df.replace(1.0, np.NaN)
    return sizes_df.apply(lambda x: pd.Series(x.dropna().values))

def finalize_and_save_results(sizes_df, number_of_nuclei_list):
    """Finalize the results, calculate metrics, and save them to Excel files."""
    sizes_df = clean_up_dataframe(sizes_df)
    sizes_df_new_nuclei = sizes_df.transpose()
    number_of_inclusions = sizes_df_new_nuclei.count(axis=1)
    average_number_of_inclusions = number_of_inclusions / number_of_nuclei_list

    excel_2 = pd.DataFrame({
        'Number_of_Inclusions': number_of_inclusions,
        'Number_of_Nuclei': number_of_nuclei_list,
        'Average_Number_of_Inclusions_per_Cell': average_number_of_inclusions
    })

    excel_2.to_excel("SIZES.xlsx")

Define Main Function

In [3]:
def main(folder_path):
    """Main function to process images and save results."""
    image_files = load_images_from_folder(folder_path)
    sizes_df = pd.DataFrame()
    number_of_nuclei_list = []

    for iteration, image_path in enumerate(image_files, start=1):
        image = imread(image_path)

        # Process DAPI channel to count nuclei
        blurred_dapi = preprocess_dapi_channel(image)
        labeled_nuclei = label(segment_nuclei(blurred_dapi))
        n_nuclei = count_nuclei(labeled_nuclei)
        number_of_nuclei_list.append(n_nuclei)

        # Process green channel to count inclusions
        confocal_img = preprocess_green_channel(image)
        labeled_inclusions = label(segment_inclusions(confocal_img))
        inclusion_sizes = measure_inclusion_sizes(labeled_inclusions, confocal_img)
        sizes_df = append_to_results(inclusion_sizes, sizes_df, image_path, iteration)

    finalize_and_save_results(sizes_df, number_of_nuclei_list)

# Run the main function on the folder with images
main("test_images")